In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from random import shuffle

from keras import Input, Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Concatenate
from keras.optimizers import RMSprop
from keras.utils import plot_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
training_data = np.load('/content/drive/My Drive/Colab Notebooks/data/training_data.npy',allow_pickle=True)
shuffle(training_data)
X = np.array([i[0] for i in training_data]).reshape(-1,96,220,1)
X = X / 255

s = np.array([i[1] for i in training_data])
MaxSpeed = max(s)
Y = np.array([(i[1]/ MaxSpeed, i[2], i[3]) for i in training_data])

del s
del training_data

In [ ]:
# model inputs
input1 = Input( shape=(96, 220, 1))
input2 = Input( shape=(48, 110, 1))

in1 = Conv2D(24, (5,5), activation='relu', padding='same')(input1)
in1 = MaxPool2D((2,2))(in1)
in1 = Conv2D(36, (5,5), activation='relu', padding='same')(in1)
in1 = MaxPool2D((2,2))(in1)
in1 = Conv2D(48, (5,5), activation='relu', padding='same')(in1)
in1 = MaxPool2D((2,2))(in1)
in1 = Conv2D(64, (3,3), activation='relu', padding='same')(in1)
in1 = MaxPool2D((2,2))(in1)
in1 = Dropout(0.25)(in1)

# In order to use Navigation data Concatenate this NAVI branch with Cam branch
# in2 = Conv2D(24, (5,5), activation='relu', padding='same')(input2)
# in2 = MaxPool2D((2,2))(in2)
# in2 = Conv2D(36, (5,5), activation='relu', padding='same')(in2)
# in2 = MaxPool2D((2,2))(in2)
# in2 = Conv2D(48, (5,5), activation='relu', padding='same')(in2)
# in2 = MaxPool2D((2,2))(in2)
# in2 = Conv2D(64, (3,3), activation='relu', padding='same')(in2)
# in2 = Dropout(0.25)(in2)
# out = Concatenate(axis=-1)([in1, in2])

out = Flatten()(in1)
out = Dense(1024, activation='relu')(out)
out = Dense(512,  activation='relu')(out)
out = Dense(256,  activation='relu')(out)
out = Dense(16,   activation='relu')(out)
out = Dense(3,    activation='sigmoid')(out)

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model = Model(inputs=input1, outputs=out)
model.compile( optimizer = optimizer,
                loss = "mse",)

In [ ]:
plot_model(model)

In [ ]:
# Model training
history = model.fit(X,
                    Y,
                    validation_split=0.1,
                    epochs = 500,
                    batch_size = 128)

In [ ]:
# model saving
model.save('/content/drive/My Drive/Colab Notebooks/models/modelX.h5')

In [ ]:
# model loading
model = load_model('/content/drive/My Drive/Colab Notebooks/models/modelX.h5')

In [ ]:
# model prediction
ID = 1412
pred = model.predict(X[ID:ID+1])
print(pred)
print(Y[ID])